# 🖼️ Retrieval-Augmented Generation (RAG) System for The Dalí Museum

This notebook demonstrates a complete RAG pipeline using:
- **OpenAI Chat Models**
- **OpenAI Embeddings**
- **ChromaDB** as a vector store
- A curated dataset about **The Dalí Museum**

The assistant retrieves relevant museum information from the vector database
and generates grounded answers using an LLM.



In [1]:
!pip install openai chromadb tqdm sentence-transformers --quiet


## 1. Installation and Setup

We install required dependencies and securely load the OpenAI API key.
This key allows access to embedding models and chat models used in this notebook.


In [2]:
import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")

if os.environ.get("OPENAI_API_KEY"):
    print("API key loaded successfully ✔️")
else:
    print("API key NOT loaded ❌")


Enter your OpenAI API key: ··········
API key loaded successfully ✔️


## 2. Dataset Description

We construct a small dataset describing various aspects of **The Dalí Museum**, including:

- Museum overview  
- Visitor information  
- Opening hours  
- Exhibitions  
- Physical location  

Each item has:
- `id`
- `title`
- `section`
- `text`  

These text entries will be converted into embeddings and indexed in ChromaDB.


In [3]:
documents = [
    {
        "id": "1",
        "title": "About the Dalí Museum",
        "section": "overview",
        "text": (
            "The Dalí Museum is located in St. Petersburg, Florida. "
            "It is dedicated to the works of the Spanish surrealist artist Salvador Dalí. "
            "The museum holds one of the largest collections of Dalí’s works outside Europe."
        ),
    },
    {
        "id": "2",
        "title": "Visitor Information",
        "section": "visit",
        "text": (
            "The museum offers guided tours, audio guides, educational programs, "
            "a garden inspired by Dalí’s work, and a museum shop with books and souvenirs."
        ),
    },
    {
        "id": "3",
        "title": "Tickets and Hours",
        "section": "tickets",
        "text": (
            "The Dalí Museum is open daily from 10 AM to 6 PM. "
            "Tickets can be purchased online or at the entrance. "
            "Discounts are available for students and seniors."
        ),
    },
    {
        "id": "4",
        "title": "Exhibitions",
        "section": "exhibitions",
        "text": (
            "The museum hosts rotating exhibitions showcasing Dalí’s paintings, drawings, "
            "sculptures, and interactive displays."
        ),
    },
    {
        "id": "5",
        "title": "Location",
        "section": "location",
        "text": (
            "The museum is located at 1 Dalí Boulevard, St. Petersburg, Florida, "
            "near the city waterfront."
        ),
    },
]

print(f"Loaded {len(documents)} documents.")


Loaded 5 documents.


## 3. Embeddings and Vector Store

We use **OpenAI's text-embedding-3-small model** to convert each document
into numerical vectors. These vectors are then stored in **ChromaDB**, enabling:

- Fast semantic search  
- Retrieval of top relevant documents  
- Grounded context for LLM responses  


In [4]:
from openai import OpenAI
import chromadb
from chromadb.utils import embedding_functions

client = OpenAI()

embedding_fn = embedding_functions.OpenAIEmbeddingFunction(
    api_key=os.environ["OPENAI_API_KEY"],
    model_name="text-embedding-3-small"
)

chroma_client = chromadb.Client()

# Remove old collection if exists
try:
    chroma_client.delete_collection("dali_museum")
except:
    pass

collection = chroma_client.create_collection(
    name="dali_museum",
    embedding_function=embedding_fn
)

print("ChromaDB collection created successfully.")


ChromaDB collection created successfully.


## 4. Inserting Documents into Vector Store

Each document is embedded and stored in ChromaDB along with metadata.
This allows the retrieval system to search semantically across all entries.


In [5]:
ids = [doc["id"] for doc in documents]
texts = [doc["text"] for doc in documents]
metadatas = [{"title": doc["title"], "section": doc["section"]} for doc in documents]

collection.add(
    ids=ids,
    documents=texts,
    metadatas=metadatas
)

print("Documents inserted into ChromaDB.")


Documents inserted into ChromaDB.


## 5. Retrieval Pipeline

We now define a function that:
1. Embeds the user query  
2. Searches the vector store  
3. Returns the top relevant text chunks  


In [6]:
def rag_search(query: str, k: int = 2):
    """Retrieve top-k documents from ChromaDB."""
    results = collection.query(query_texts=[query], n_results=k)
    return results


## 6. Answer Generation with LLM

The retrieved context is passed to OpenAI's chat model (`gpt-4o-mini`).
The LLM must ground its answer ONLY on the retrieved context.


In [7]:
def rag_search(query: str, k: int = 2):
    """Retrieve top-k documents from ChromaDB."""
    results = collection.query(query_texts=[query], n_results=k)
    return results


## 6. Answer Generation with LLM

The retrieved context is passed to OpenAI's chat model (`gpt-4o-mini`).
The LLM must ground its answer ONLY on the retrieved context.


In [8]:
def llm_answer(question: str, context: str) -> str:
    """Generate answer using OpenAI LLM with retrieved context."""

    prompt = f"""
You are a helpful museum assistant specializing in Salvador Dalí and The Dalí Museum.
Answer the question using only the information provided in the context.
If the answer is not present, reply: 'The context does not contain this information.'

CONTEXT:
{context}

QUESTION:
{question}

ANSWER:
"""

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )

    return response.choices[0].message.content.strip()


## 7. Full RAG Pipeline

This combines:
- Semantic search  
- Context assembling  
- LLM answer generation  

to produce a grounded, reliable response.


In [9]:
def rag_answer(question: str, k: int = 2):
    """Run full RAG pipeline."""

    search_results = rag_search(question, k)
    context_chunks = search_results["documents"][0]
    context = "\n\n".join(context_chunks)

    print("=== Retrieved Context ===")
    print(context)
    print("\n=== Answer ===")

    return llm_answer(question, context)


## 8. Testing the RAG System

Let's test the RAG pipeline with common museum questions.


In [10]:
rag_answer("Where is the Dalí Museum located?")
rag_answer("What are the opening hours?")
rag_answer("Does the museum offer guided tours?")


=== Retrieved Context ===
The Dalí Museum is open daily from 10 AM to 6 PM. Tickets can be purchased online or at the entrance. Discounts are available for students and seniors.

The Dalí Museum is located in St. Petersburg, Florida. It is dedicated to the works of the Spanish surrealist artist Salvador Dalí. The museum holds one of the largest collections of Dalí’s works outside Europe.

=== Answer ===
=== Retrieved Context ===
The Dalí Museum is open daily from 10 AM to 6 PM. Tickets can be purchased online or at the entrance. Discounts are available for students and seniors.

The museum hosts rotating exhibitions showcasing Dalí’s paintings, drawings, sculptures, and interactive displays.

=== Answer ===
=== Retrieved Context ===
The museum offers guided tours, audio guides, educational programs, a garden inspired by Dalí’s work, and a museum shop with books and souvenirs.

The Dalí Museum is open daily from 10 AM to 6 PM. Tickets can be purchased online or at the entrance. Discount

'Yes, the museum offers guided tours.'